In [ ]:
import sys
import os

os.chdir("..")
os.chdir("./src")
# sys.path.append("./src")

In [ ]:
from IPython import display
from IPython.display import clear_output
import pylab as pl
import numpy as np
import matplotlib.pyplot as plt
from numba import njit, jit
from time import time

# from helpers import *
from LDMIBSS import *

# np.random.seed(14)
%load_ext autoreload
%autoreload 2

In [ ]:
N = 50000
NumberofSources = 5
NumberofMixtures = 8
S = generate_correlated_copula_sources(
    rho=0.8,
    df=4,
    n_sources=NumberofSources,
    size_sources=N,
    decreasing_correlation=False,
)
S = 2 * S - 1
print("The following is the correlation matrix of sources")
display_matrix(np.corrcoef(S))

# Generate Mxr random mixing from i.i.d N(0,1)
A = np.random.randn(NumberofMixtures, NumberofSources)
X = np.dot(A, S)

SNR = 30  # dB
NoiseAmp = 10 ** (-SNR / 20)  # * np.sqrt(NumberofSources)
NoisePart = NoiseAmp * np.random.randn(X.shape[0], X.shape[1])
X = X + NoisePart
SNRinp = 10 * np.log10(np.sum(np.var(X, axis=1)) / np.sum(np.var(NoisePart, axis=1)))
print("The following is the mixture matrix A")
display_matrix(A)
print("Input SNR is : {}".format(SNRinp))

In [ ]:
s_dim = S.shape[0]
x_dim = X.shape[0]
debug_iteration_point = 200
model = LDMIBSS(s_dim=s_dim, x_dim=x_dim, set_ground_truth=True, S=S, A=A)

In [ ]:
model.fit_batch_antisparse(
    X,
    epsilon=1e-5,
    mu_start=200,
    n_iterations=10000,
    method="covariance",
    debug_iteration_point=debug_iteration_point,
    plot_in_jupyter=True,
)

In [ ]:
W = model.W
Y = W @ X
10 * np.log10(CalculateSINR(Y, S)[0])

In [ ]:
CalculateSIR(A, W)

In [ ]:
Y = W @ X
Y.shape, X.shape, S.shape
Y_ = signed_and_permutation_corrected_sources(S.T, Y.T).T
snr(S.T, Y_.T)

In [ ]:
snr(S.reshape(-1, 1), Y_.reshape(-1, 1))